In [3]:
import requests
from bs4 import BeautifulSoup
import csv
import time

BASE_URL = "https://www.huila.gov.co"
LISTADO_URL = BASE_URL + "/cultura-y-turismo/publicaciones/noticias/?tema=15&genPag={}"

# Cabeceras para evitar bloqueos
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

# Archivo CSV
OUTPUT_FILE = "noticias_huila.csv"

def extraer_detalle(url):
    """Extrae el contenido completo de una noticia desde su página interna."""
    try:
        resp = requests.get(url, headers=HEADERS, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")

        # Contenido de la noticia
        contenedor = soup.select_one("div.publication-container .pgel")
        if contenedor:
            parrafos = contenedor.find_all("p")
            contenido = " ".join([p.get_text(" ", strip=True) for p in parrafos])
        else:
            contenido = None

        return contenido
    except Exception as e:
        print(f"❌ Error en {url}: {e}")
        return None

def extraer_listado():
    noticias = []

    for pagina in range(1, 13):  # páginas 1 a 12
        url = LISTADO_URL.format(pagina)
        print(f"🔎 Procesando {url}...")
        try:
            resp = requests.get(url, headers=HEADERS, timeout=10)
            resp.raise_for_status()
            soup = BeautifulSoup(resp.text, "html.parser")

            # Iterar sobre cada noticia
            for h2 in soup.select("div.modContent h2"):
                link_tag = h2.find("a")
                titulo = link_tag.get_text(strip=True) if link_tag else None
                enlace = link_tag["href"] if link_tag else None
                if enlace and not enlace.startswith("http"):
                    enlace = BASE_URL + enlace

                # Imagen y descripción (están en el siguiente div.row)
                row = h2.find_next_sibling("div", class_="row")
                if row:
                    img_tag = row.find("img")
                    imagen = BASE_URL + img_tag["src"] if img_tag else None
                    desc_tag = row.find("p", class_="pgel desc-cat-doc")
                    descripcion = desc_tag.get_text(strip=True) if desc_tag else None
                else:
                    imagen, descripcion = None, None

                # Fecha (está después del row)
                fecha_tag = row.find_next_sibling("div", class_="pubTema text-right") if row else None
                fecha = fecha_tag.get_text(strip=True) if fecha_tag else None

                # Extraer contenido interno
                contenido = extraer_detalle(enlace) if enlace else None

                noticias.append({
                    "titulo": titulo,
                    "enlace": enlace,
                    "imagen": imagen,
                    "descripcion": descripcion,
                    "fecha": fecha,
                    "contenido": contenido
                })

                time.sleep(1)  # respetar al servidor

        except Exception as e:
            print(f"⚠️ Error en la página {pagina}: {e}")

    return noticias

if __name__ == "__main__":
    datos = extraer_listado()

    # Guardar en CSV
    with open(OUTPUT_FILE, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["titulo", "enlace", "imagen", "descripcion", "fecha", "contenido"])
        writer.writeheader()
        writer.writerows(datos)

    print(f"✅ Listo! Se guardaron {len(datos)} noticias en {OUTPUT_FILE}")


🔎 Procesando https://www.huila.gov.co/cultura-y-turismo/publicaciones/noticias/?tema=15&genPag=1...
🔎 Procesando https://www.huila.gov.co/cultura-y-turismo/publicaciones/noticias/?tema=15&genPag=2...
🔎 Procesando https://www.huila.gov.co/cultura-y-turismo/publicaciones/noticias/?tema=15&genPag=3...
🔎 Procesando https://www.huila.gov.co/cultura-y-turismo/publicaciones/noticias/?tema=15&genPag=4...
🔎 Procesando https://www.huila.gov.co/cultura-y-turismo/publicaciones/noticias/?tema=15&genPag=5...
🔎 Procesando https://www.huila.gov.co/cultura-y-turismo/publicaciones/noticias/?tema=15&genPag=6...
🔎 Procesando https://www.huila.gov.co/cultura-y-turismo/publicaciones/noticias/?tema=15&genPag=7...
🔎 Procesando https://www.huila.gov.co/cultura-y-turismo/publicaciones/noticias/?tema=15&genPag=8...
🔎 Procesando https://www.huila.gov.co/cultura-y-turismo/publicaciones/noticias/?tema=15&genPag=9...
🔎 Procesando https://www.huila.gov.co/cultura-y-turismo/publicaciones/noticias/?tema=15&genPag=10...